In [2]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 15

# 8.1 Hierarchical Indexing

In [3]:
# hierachical indexing is an important feature of pandas that enables you to have multiple index levels on an axis
# it provides a way for you to work with higher dimensional data in a lower dimensional form

# create data
# pass in a list of lists (or arrays) as an index to create hierarchical indexing
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]]) 

data

a  1    1.007972
   2    1.397220
   3    0.831837
b  1    0.117745
   3   -0.104404
c  1   -0.742172
   2    1.091319
d  2    1.818605
   3   -0.666408
dtype: float64

In [4]:
# this type of Series has a 'MultiIndex object' as its index

data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
# hierarchical indexed objects enable you to concisely select subsets of the data

print(data['b'])
print('----------------------')
print(data['b':'c'])
print('----------------------')
print(data.loc[['b', 'd']])
print('----------------------')
print(data.loc[:, 2]) # selection is even possible from an "inner" level

1    0.117745
3   -0.104404
dtype: float64
----------------------
b  1    0.117745
   3   -0.104404
c  1   -0.742172
   2    1.091319
dtype: float64
----------------------
b  1    0.117745
   3   -0.104404
d  2    1.818605
   3   -0.666408
dtype: float64
----------------------
a    1.397220
c    1.091319
d    1.818605
dtype: float64


In [ ]:
###

In [6]:
# hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table

# you could rearrange the data into a DataFrame using its unstack() method
data.unstack()

# Unstack the prescribed level from index to columns
# the new inner-most levels are created by pivoting the index of the current DataFrame
# if the index is not a MultiIndex, the output will be a Series, else the output will be a DataFrame

,1,2,3
a,1.007972,1.397220,0.831837
b,0.117745,NaN,-0.104404
c,-0.742172,1.091319,NaN
d,NaN,1.818605,-0.666408


In [7]:
# the inverse operation of unstack() is stack()
data.unstack().stack()

# Stack the prescribed levels from columns to index
# the new inner-most levels are created by pivoting the columns of the current DataFrame
# if the column is not a MultiIndex, the output will be a Series, else the output will be a DataFrame
# Series has no stack() method

a  1    1.007972
   2    1.397220
   3    0.831837
b  1    0.117745
   3   -0.104404
c  1   -0.742172
   2    1.091319
d  2    1.818605
   3   -0.666408
dtype: float64

In [8]:
# with a DataFrame, either axis can have a hierarchical index

# create data
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [9]:
# the hierarchical levels can have names

frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [10]:
# with partial column indexing, you can similarly select groups of columns

frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [11]:
# a MultiIndex can be created by itself and then reused

# for example, the columns of the preceding DataFrame could be created like this
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## 8.1.1 Reordering and Sorting Levels

In [12]:
# the swaplevel() takes two level numbers or names and returns a new object with the levels interchanged

frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [13]:
# the sort_index() sorts the data using only the values in a single level

frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [14]:
# commonly, sort_index() and swaplevel() are used together so that the result is lexicographically sorted by the indicated level

frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 8.1.2 Summary Statistics by Level

In [15]:
# many descriptive and summary statistics on DataFrame and Series have a 'level' option
# it allows you to specify the level you want to aggregate by on a particular axis

# frame.sum(level='key2') # deprecated

# on newer versions, the 'level' option has been move to the df.groupby() function
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.groupby(level='color', axis=1).sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 8.1.3 Indexing with a DataFrame's columns

In [17]:
# Example when you want to use one or more columns from a DataFrame as the row index

# create data
frame = pd.DataFrame(
    {
        'a' : range(7), 
        'b' : range(7, 0, -1),
        'c' : ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
        'd' : [0, 1, 2, 0, 1, 2, 3]
    }
)

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [18]:
# DataFrame's set_index() function will create a new DataFrame using one or more of its columns as the index

frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [19]:
# by default, the index columns are removed from the DataFrame, but you can keep them using the 'drop' option

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [20]:
# reset_index() does the opposite (it moves the hierarchical index levels into the columns)

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

**Data contained in pandas objects can be combined together in a number of ways:**

---

- **pandas.merge** connects rows in DataFrames based on one or more keys.
- **pandas.concat** concatenates or "stack" together objects along an axis
- the **combine_first** instance method enable splicing together overlapping data to fill in missing values in one object with values from another

## 8.2.1 Database-Style DataFrame Joins

In [21]:
# Merge or Join operations combine datasets by linking rows using one or more keys
# the merge() function in pandas is the main entry point for using these algorithms on your data

# create data
df1 = pd.DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1' : range(7)})

# display data
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
# create data
df2 = pd.DataFrame({'key' : ['a', 'b', 'd'],
                    'data2' : range(3)})

# display data
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [23]:
# an example of a many-to-one join
# df1 has 'many' rows labeled a and b, df2 has 'one' row for each value in the key column

pd.merge(df1, df2, on='key') # the 'on' option specify which column is used to merge datas

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [24]:
# create data
df3 = pd.DataFrame({'lkey' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1' : range(7)})

# display data
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
# create data
df4 = pd.DataFrame({'rkey' : ['a', 'b', 'd'],
                    'data2' : range(3)})

# display data
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [26]:
# if the column names are different in each object, you can specify them seperately
# using the 'left_on' and 'right_on' arguments

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [27]:
# you may notice that the 'c' and 'd' values and associated data are missing from the result
# that is because by default, merge() does an 'inner' join (other possible options are: left, right, and outer)

pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [28]:
# create data
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                    'data1': range(6)})

# display data
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [29]:
# create data
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

# display data
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [30]:
# Many-to-many merges have well-defined, though not necessarily intuitive behavior
# it joins form the Cartesian product of the rows

pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [31]:
# the join method only affects the distinct key values appearing in the result

pd.merge(df1, df2, on='key', how='inner') # still the same values, this time with no 'c' key

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [32]:
# create data
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})

# display data
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [33]:
# create data
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                    'key2': ['one', 'one', 'one', 'two'],
                    'rval': [4, 5, 6, 7]})

# display data
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [34]:
# to merge with multiple keys, pass a list of column names to 'on'

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [35]:
# A last issue in merge operations is the treatment of overlapping column names
# merge() has a 'suffixes' option for specifying strings to append to overlapping names in the left and right df

pd.merge(left, right, on='key1') # by default, left is '_x' and right is '_y'

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [36]:
# you can also specify your own suffixes

pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 8.2.2 Merging on Index

In [37]:
# in some cases, the merge key(s) in DataFrame will be found in its index
# in this case, you can pass left_index=True or right_index=True or both 
# to indicate that the index should be used as the merge key

# create data
left1 = pd.DataFrame({'key' : ['a', 'b', 'a', 'a', 'b', 'c'],
                    'value' : range(6)})

# display data
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
# create data
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

# display data
right1


,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [40]:
# since the default merge method is to intersect the join keys, you can instead form the union with an outer join

pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [41]:
# with hierarchically indexed data, things are more complicated
# as joining on index is implicitly a multiple-key merge

# create data
lefth = pd.DataFrame({'key1' : ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                     'key2' : [2000, 2001, 2002, 2001, 2002],
                     'data' : np.arange(5.)})

# display data
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [42]:
# create data
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

# display data
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [43]:
# in this case, you have to indicate multiple columns to merge on as a list

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [44]:
# merge but use how='outer' to handle duplicate values

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [45]:
# create data
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], 
                     index=['a', 'c', 'e'], columns=['Ohio', 'Nevada'])

# display data
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
# create data
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
# display data
right2


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [47]:
# using the indexes of both sides of the merge is also possible

pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [48]:
# dataframe has a join() instance for merging by index

left2.join(right2, how='outer') 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [49]:
left1.join(right1, on='key') # left1 use 'key', right1 use index

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [50]:
# create data
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'], 
                       columns=['New York', 'Oregon'])

# display data
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [51]:
# for index-on-index merges, you can pass a list of DataFrame to join()

left2.join([right2, another], how='outer').sort_index()

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## 8.2.3 Concatenating Along an Axis

In [52]:
# another kind of data manipulation is called concatenating (or binding, stacking)
# Numpy's concatenate() function can do this with Numpy arrays

# create data
arr = np.arange(12).reshape((3, 4))

# display data
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [53]:
np.concatenate([arr, arr], axis=1) # horizontal concatenation

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [54]:
# the concat() function in Pandas helps you concatenate Series and DataFrame objects

# create data
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [55]:
# calling concat() with a list of these objects in a list glues together the values and indexes

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [56]:
# by defautlt, concat() works on axis=0, which creates a new Series
# you can modify to axis=1 to create a new DataFrame

pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [57]:
# create data
s4 = pd.concat([s1, s3])

# display data
s4

a    0
b    1
f    5
g    6
dtype: int64

In [58]:
# by default, concat() use the join='outer' (union) of the indexes

pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [59]:
# you can intersect them using join='inner'

pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [60]:
# suppose you want to create a hierarchical index on the concatenation axis. To do this, use the 'keys' argument

result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [61]:
result.unstack() # unstack a Series to turn it into a DataFrame

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [62]:
# in the case of combining Series along axis=1, the keys become the DataFrame's column headers

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [63]:
# the same logic extends to DataFrame objects

# create data
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])

# display data
df1

,one,two
a,0,1
b,2,3
c,4,5


In [64]:
# create data
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

# displlay data
df2

,three,four
a,5,6
c,7,8


In [65]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [66]:
# if you pass a dict of objects instead of a list, the dict's keys will be used for the keys option

pd.concat({'level1' : df1, 'level2' : df2}, axis=1) # works the same as the above cell

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [67]:
# if the row index of a DataFrame does not contain any relevant data, you can pass ignore_index=True

# create data
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

# display data
df1

,a,b,c,d
0,0.465626,1.081717,-2.418691,-0.427473
1,1.883182,1.043532,0.375911,0.755314
2,-1.021895,-1.657658,0.890870,-0.183446


In [68]:
# create data
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['d', 'b', 'a'])

# display data
df2

,d,b,a
0,0.693485,0.930540,-0.296613
1,0.952600,0.447212,0.046552


In [69]:
pd.concat([df1, df2], ignore_index=True) #ignoring index

,a,b,c,d
0,0.465626,1.081717,-2.418691,-0.427473
1,1.883182,1.043532,0.375911,0.755314
2,-1.021895,-1.657658,0.890870,-0.183446
3,-0.296613,0.930540,NaN,0.693485
4,0.046552,0.447212,NaN,0.952600


## 8.2.4 Combining Data with Overlap

In [70]:
# you may have to process two datasets whose indexes overlap in full or part

# create data
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])

# display data
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [71]:
# create data
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

# display data
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [72]:
# Numpy's where() function perform the array-oriented equivalent of an if-else expression

np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [73]:
# Series has a combine_first() method, which performs the equivalent of this operation

b[:-2].combine_first(a[2:]) 
# since b[:-2] doesn't contain ['a', 'b'] indexes, the values are taken from a[2:]

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [74]:
# with DataFrame, combine_first() does the same thing column by column

# create data
df1 = pd.DataFrame({'a' : [1., np.nan, 5., np.nan],
                    'b' : [np.nan, 2., np.nan, 6.],
                    'c' : range(2, 18, 4)})
# display data
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [75]:
# create data
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

# display data
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [76]:
# you can think of it as "patching" missing data in the calling object with data from the object you pass
# the resulting DataFrame will be the union of the two

df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshaping and Pivoting

In [77]:
# there are a number of basic operations for rearranging tabular data
# these are alternatingly referred to as 'reshape' or 'pivot' operations

# stack: this "rotate" or pivots from the columns in the data to the rows
# unstack: this pivots from the rows into the columns

## 8.3.1 Reshaping with Hierarchical Indexing 

In [78]:
# there are two primary actions: stack() and unstack()
# unstack(): index -> column
# stack(): column -> index
# Series don't have the 'stack' method

#create data
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

# display data
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
# using the stack() method on this data pivots the columns into the rows, producing a Series

result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [80]:
# from a hierarchical indexed Series, you can rearrange the data back into a DataFrame with unstack()

result.unstack() 

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [81]:
# by default, the innermost level is stacked (unstacked). You can stack (unstack) a different level by passing a level number or name

result.unstack(0) 
# result.unstack('state') # works the same

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [82]:
# unstacking might introduce missing data if all the values in the level aren't found in each of the subgroup

# create data
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [83]:
data2.unstack() # missing data introduced

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [84]:
# stacking filters out missing data by default

data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [85]:
# you can disable missing data filter 

data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [86]:
# create data
df = pd.DataFrame({'left' : result, 'right' : result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

# create data
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
# when you unstack() a DataFrame, the level unstacked becomes the lowest level in the result

df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
# when calling stack(), we can indicate the name of the axis to stack
# the level stacked also becomes the lowest level in the result

df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 8.3.2 Pivoting "Long" to "Wide" Format

In [106]:
# A common way to store multiple time series in databases and CSV is in "long" or "stacked" format

# create data
data = pd.read_csv('examples/macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [107]:
# PeriodIndex combines the 'year' and 'quarter' columns to create a kind of time interval type
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date') # learn more in chapter 11

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns={0: 'value'})

# now ldata looks like this
ldata[:10] # long format

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [108]:
# you might prefer to have a DataFrame containing one column per distinct 'item' value indexed by timestamps in the 'date' column
# DataFrame's pivot() method performs exactly this transformation

pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
# suppose you have two value columns that you wanted to reshape simultaneously

# modify data
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-2.423708
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.720613
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.674114
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.087529
4,1959-06-30 23:59:59.999999999,infl,2.340,0.053598
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.541600
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.718106
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.570780
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.426344
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-2.201518


In [ ]:
# by omitting the last argument, you obtain a DataFrame with hierarchical columns

pivoted = ldata.pivot('date', 'item')
pivoted

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.720613 -2.423708   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  0.053598 -1.087529   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.570780  0.718106   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.048632 -2.201518   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.493293 -0.969920   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  0.605966  0.352233   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.963041  0.858646   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1 -0.657518 -0.834859   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  0.098181  0.068189   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6 -0.484716 -0.259570   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.674114  
1959-06-30 23:59:59.999999999 -0.541600  
1959-09-30 23:59:59.999999999  1.426344  
1959-12-31 23:59:59.999999999  0.717539  
1960-03-31 23:59:59.999999999  0.418533  
...                                 ...  
2008-09-30 23:59:59.999999999  1.090506  
2008-12-31 23:59:59.999999999  1.183581  
2009-03-31 23:59:59.999999999  0.011808  
2009-06-30 23:59:59.999999999  1.725012  
2009-09-30 23:59:59.999999999  0.020508  

[203 rows x 6 columns]

In [112]:
# note that pivot() is equivalent to creating a hierarchical index using set_index() followed by a call to unstack()

unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                
item                           infl   realgdp unemp
date                                               
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6

## 8.3.3 Pivoting "Wide" to "Long" Format

In [ ]:
# an inverse to pivot() for DataFrame is pandas.melt()
# rather than transforming one column into many in a new DataFrame
# it merges multiple columns into one, producing a DataFrame that is longer than the input

# create data
df = pd.DataFrame({'key' : ['foo', 'bar', 'baz'],
                   'A' : [1, 2, 3], 
                   'B' : [4, 5, 6],
                   'C' : [7, 8, 9]})

# display data
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
# the 'key' column may be a group indicator, and the other columns are data values
# when using pandas.melt(), we must indicate which columns (if any) are group indicators

melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
# using pivot(), we can reshape back to the original layout

reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
# since the result of pivot() creates an index from the column used as the row labels
# we may want to use reset_index() to move the data back into a colun

reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
# you can also specify a subset of columns to use as value columns

pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
# pandas.melt() can be used without any group identifiers, too

pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [ ]:
pd.melt(df, value_vars=['key', 'A', 'B']) 

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
